# KNR ROBOBUDEX

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import ecoinfo
from rules import *
import rasterio

test
Rule more renewable: tensor([ 0.3727, -0.5487,  0.1440])
Rule more fiber optics: tensor([ 0.0890, -0.1390,  0.0200])
Rule heating region: tensor([-3.3000,  7.3000, -3.0000])
Rule facilities: tensor([   0., -138.,    0.])
Rule possible heat: tensor([-0.4840, -6.2460,  3.0410])
Rule good solar: tensor([ 0.4870, -0.1270, -0.3000])
Rule high wind: tensor([0., 0., 0.])
Total rule score: tensor([-0.6594, -1.0254, -0.0518])


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

### DATASET & DATALOADER

In [4]:
class USADataset(Dataset):
    def __init__(self):
        self.sampling_lat = 40
        self.sampling_lon = 80 
        self.lendata = 0
        self.data = self.generate_data()
        
        

    def generate_data(self):
        latitudes = torch.linspace(25.0, 49.5, self.sampling_lat)
        longtitudes = torch.linspace(-125.0, -67.0, self.sampling_lon)
        data = torch.zeros((self.sampling_lat * self.sampling_lon, 7), dtype=torch.float32)
        sample_idx = 0
        raster_map = ecoinfo.get_default_params_map()

        df_data = ecoinfo.extract_multiple_to_df(raster_map, [(lat, lon) for lat in latitudes for lon in longtitudes])
        while sample_idx < len(df_data):
            row = df_data.iloc[sample_idx]
            data[sample_idx, 0:] = torch.tensor([
                row["lat"],
                row["lon"],  
                row["Wind efficiency"],
                row["Solar power"],
                row["Fiber optics"],
                row["Temperature"],
                row["Popilation density"]
            ], dtype=torch.float32)
            sample_idx += 1

        self.lendata = sample_idx
        return data
                
                    
        
    def __len__(self):
        return self.lendata

    def __getitem__(self, idx):
        return self.data[idx,2:]
        
dataset = USADataset()      


In [13]:
dataloader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=True
)
print(f"DATASET SHAPE: {len(dataset)}")

DATASET SHAPE: 1779


### MODEL

In [31]:
input_dim = 5
lr = 1e-3
epochs = 100

In [32]:
def ranknet_loss(score_a, score_b, S):
    """
    score_a, score_b: predicted scores for pairs
    S: target preference (+1 if a > b, -1 if a < b)
    """
    pred = torch.sigmoid(score_a - score_b)
    target = (S + 1) / 2
    loss = nn.BCELoss()(pred, target, )
    return loss


def generate_pairs(batch):
    """
    Generates all pairs from a batch (a,b) for ranking.
    """
    xa = batch.unsqueeze(1).repeat(1, batch.size(0), 1)  # [B,B,F]
    xb = batch.unsqueeze(0).repeat(batch.size(0), 1, 1)  # [B,B,F]

    # Only take upper triangular pairs to avoid duplicates and self-pairs
    mask = torch.triu(torch.ones(xa.shape[0], xa.shape[1]), diagonal=1).bool()
    xa_pairs = xa[mask]
    xb_pairs = xb[mask]
    return xa_pairs, xb_pairs



In [33]:
class RankNet(nn.Module):
    def __init__(self, input_dim):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
        
    def forward(self, x):
        return self.model(x)

model = RankNet(input_dim=input_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)


for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        
        xa, xb = generate_pairs(batch)
        
        S = total_rule_score(xa, xb)
        S = S.unsqueeze(1)
        S_binary = torch.sign(S).clamp(min=-1.0, max=1.0)
        
        score_a = model(xa)
        score_b = model(xb)
        
        loss = ranknet_loss(score_a, score_b, S_binary)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    scheduler.step()

    if (epoch%10 == 0):
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/100, Loss: 0.7033
Epoch 11/100, Loss: 0.4812
Epoch 21/100, Loss: 0.4162
Epoch 31/100, Loss: 0.3950
Epoch 41/100, Loss: 0.4504
Epoch 51/100, Loss: 0.4132
Epoch 61/100, Loss: 0.4059
Epoch 71/100, Loss: 0.4256
Epoch 81/100, Loss: 0.4065
Epoch 91/100, Loss: 0.4078
